In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from ssl import create_default_context
import requests
from getpass import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime
from datetime import timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [2]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy, es usada para concatenar al nombre del indice principal previa inserción
* ahora, es la fecha en un formato requerido para ES

In [3]:
now = datetime.now()
format_ES = "%Y.%m.%d"
fecha_hoy = str(now.strftime(format_ES))
ahora_format = "%Y-%m-%d"'T'"%H:%M:%S"
ahora = str(now.strftime(ahora_format))

### Definiendo indice principal con fecha de hoy

In [4]:
indice = parametros.gestion_mintic_concat_index 
#indice

### Función para generar JSON compatible con ES

In [60]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [61]:
total_docs = 1
try:
    response = es.search(
        index= parametros.mintic_control,
        body={
           "_source": ["gestion.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_gestion_interacciones"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {filterKeys}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['gestion.fechaControl']
except Exception as e:
    print(e)
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-14T00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)



ultima fecha para control de ejecucion: 2021-07-01T18:00:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede.)

In [5]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
                           ,'nombreSede','latitud', 'longitud','id_Beneficiario','COD_ISO','codDanesede',
                           'cod_servicio','codDaneMuni','nombre_centro_pob','codCentroPoblado','codDaneInstitucionEdu',
                           'tipoSitio','detalleSitio','energia','region','matricula','DDA','grupoDesc','estadoInstalacion',
                           'nombreInstitucionEd']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except Exception as e:
    print(e)
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()

In [6]:
datos_semilla

,nombreSede,DDA,estadoInstalacion,latitud,COD_ISO,energia,nombre_departamento,codCentroPoblado,longitud,nombreInstitucionEd,...,site_id,matricula,energiadesc,grupoDesc,cod_servicio,region,detalleSitio,nombre_municipio,id_Beneficiario,codDaneInstitucionEdu
0,C.E.R LA PAULINA,5,4,"6,7048548",CO-ANT,1,ANTIOQUIA,333.0,"-76,14525469",C. E. R. LA PAULINA,...,22154-ZGYO176,111,RED INTERCONECTADA,OBLIGATORIOS,ZGYO176,A,8,VALDIVIA,22154,205854000501
1,CENTRO EDUCATIVO RURAL EL CERRO,5,4,"4,225166667",CO-ANT,1,ANTIOQUIA,333.0,"-73,10716667",I. E. MANUEL ANTONIO TORO,...,20022-ZGYO177,22,RED INTERCONECTADA,OBLIGATORIOS,ZGYO177,A,8,FRONTINO,20022,105284000214
2,C. E. R. MERCEDES YEPES,5,1,"6,93848274",CO-ANT,1,ANTIOQUIA,333.0,"-75,151177190",C. E. R. EL CARMIN,...,,42,RED INTERCONECTADA,OBLIGATORIOS,,A,8,ANORÍ,18787,205040000357
3,C. E. R. MEDIAS FALDAS,5,1,"7,04797951",CO-ANT,4,ANTIOQUIA,333.0,"-75,19442611",I. E. ANORI,...,,84,TIENE ENERGÍA,Opción 2,,A,8,ANORÍ,18802,105040000212
4,C. E. R. ROSAURA ORTEGA,5,2,"7,06272584",CO-ANT,1,ANTIOQUIA,333.0,"-75,22730972",C. E. R. EL CARMIN,...,,14,RED INTERCONECTADA,OBLIGATORIOS,,A,8,ANORÍ,18804,205040000357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6942,ESCUELA SANTA ISABEL,5,1,"6,39026335",CO-VAU,3,VAUPÉS,,"-73,138349330",I.E.D. DE PIEDRA ÑI,...,,53,SOLUCIÓN SOLAR,OBLIGATORIOS,,A,8,PACOA,67733,297001001879
6943,ESCUELA RURAL DE SAN LUIS DE PIEDRA ÑI,5,1,"0,08164385",CO-VAU,4,VAUPÉS,,"-70,07416384",I.E.D. DE PIEDRA ÑI,...,,14,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70622,297001001879
6944,ESCUELA RURAL DE SANTA ROSA,5,1,,CO-VAU,4,VAUPÉS,,,I.E.D. DE PIEDRA ÑI,...,,36,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70623,297001001879
6945,ESCUELA RURAL DE TOAKA,5,1,"1,007644476",CO-VAU,4,VAUPÉS,,"-71,25103723",I.E.D. DE PIEDRA ÑI,...,,32,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70624,297001001879


In [63]:
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['gestion.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['gestion.location']=datos_semilla['gestion.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)




### Leyendo indice servicemanager-interacciones

Se lee la información de los De Service Manager. En la lectura se traen todas las interacciones reportadas por servicemanager, con la información de la misma.

* id_beneficiario, es la llave para cruzar con cada centro de conexión.
* variable1 y variable2, son la tipo y subtipo de la interaccion.
* current_phase, estado del ticket
* clr_txt_assignment, es el sujeto asignado al ticket
* owner_name, usuario del ticket
* open_time y close_time, son las fecha de apertura y cierre del ticket
* resolution, respuesta al ticket
* incident_id, numero del ticket
* source, fuente del ticket(email o telefono)
* severity, gravedad del ticket
* clr_txt_codcategorymtc categoria segun mintic

In [46]:
def traeSMInteracciones(fecha_max,fecha_tope):
    total_docs = 10000
    response = es.search(
        index= parametros.sm_interaciones,
        body={
            "_source": ["clr_txt_idbneficmtc","variable1","variable2","variable3"
                        ,"variable4","current_phase","clr_txt_assignment","owner_name"
                        ,"open_time","close_time","resolution","open","incident_id","source","severity"]
            ,"query": {
              "range": {
                "open_time": {
                    "gte": fecha_max,
                    "lt": fecha_tope
                }
              }
          }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_SM_interactions = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    
    if datos_SM_interactions is None or datos_SM_interactions.empty:
        return pd.DataFrame()
    datos_SM_interactions = datos_SM_interactions.rename(columns={'severity' : 'gestion.gravedad'
                                                                  ,'variable1' : 'gestion.categoria'
                                                                  ,'variable2' : 'gestion.subcategoria'
                                                                  ,'current_phase':'gestion.estadoTicket'
                                                                  ,'owner_name':'gestion.usuarioTicket'
                                                                  ,'clr_txt_assignment':'gestion.responsable'
                                                                  ,'source':'gestion.canal'
                                                                  ,'clr_txt_idbneficmtc':'id_Beneficiario'
                                                                  ,'incident_id':'gestion.numeroTicket'})         
    datos_SM_interactions = datos_SM_interactions.dropna(subset=['id_Beneficiario'])
    datos_SM_interactions = datos_SM_interactions.drop(datos_SM_interactions[datos_SM_interactions['id_Beneficiario']=="null"].index)
    datos_SM_interactions['id_Beneficiario'] = datos_SM_interactions['id_Beneficiario'].astype(int)
    datos_SM_interactions['gestion.canal'] = datos_SM_interactions['gestion.canal'].replace(['5','6'],['Correo Electrónico','Teléfono'])
    datos_SM_interactions['gestion.gravedad'] = datos_SM_interactions['gestion.gravedad'].replace(['1','2','3'],['Alto','Medio','Bajo'])

    #datos_SM_interactions['gestion.tiempoRespuesta'] = datos_SM_interactions.apply(lambda row: row.close_time - row.open_time, axis=1)

    return datos_SM_interactions





Realizando bucle hasta conseguir datos de servicemanager-interacciones o hasta la fecha actual para realizar la carga de datos 

In [47]:
fecha_max_mintic = fecha_ejecucion



fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
datos_SM_interactions = traeSMInteracciones(fecha_max_mintic,fecha_tope_mintic)


if datos_SM_interactions is None or datos_SM_interactions.empty:
    while (datos_SM_interactions is None or datos_SM_interactions.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d"'T'"%H:%M:%S")) < str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        datos_SM_interactions = traeSMInteracciones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass



In [48]:
#fecha_max_mintic
#print (fecha_max_mintic, "   hastaaaa  ", fecha_tope_mintic)

Haciendo merge entre semilla e interacciones

Realizando inserción

In [49]:
use_these_keys = ['gestion.numeroTicket'
                  , 'gestion.usuarioTicket'
                  , 'gestion.estadoTicket'
                  , 'gestion.categoria'
                  , 'gestion.subcategoria'
                  , 'gestion.fechaApertura'
                  , 'gestion.fechaCierre'
                  , 'gestion.responsable'
                  , 'gestion.location'
                  , 'gestion.canal'
                  , 'gestion.id_Beneficiario'
                  , 'gestion.resolucion'
                  , 'gestion.Tipo3'
                  , 'gestion.Tipo4'
                  , 'gestion.nombreInstitucionEd'
                  , 'gestion.abierto'
                  , 'gestion.nombreSede'
                  , 'gestion.COD_ISO'
                  , 'gestion.site_id'
                  , 'gestion.dptoGestion'
                  , 'gestion.muniGestion'
                  , 'gestion.energiadesc'
                  , 'gestion.gravedad'
                  , 'gestion.anyo'
                  , 'gestion.mes'
                  , 'gestion.dia'
                  , 'gestion.hora'
                  , 'gestion.min'
                  , 'gestion.seg'
                  , 'gestion.cod_servicio'
                  , 'gestion.codDaneMuni'
                  , 'gestion.nombre_centro_pob'
                  , 'gestion.codCentroPoblado'
                  , 'gestion.codDaneInstitucionEdu'
                  , 'gestion.tipoSitio'
                  , 'gestion.detalleSitio'
                  , 'gestion.energia'
                  , 'gestion.region'
                  , 'gestion.matricula'
                  , 'gestion.DDA'
                  , 'gestion.grupoDesc'
                  , 'gestion.estadoInstalacion'
                  , '@timestamp']
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{str(document['gestion.site_id']) + '-' + str(document['gestion.numeroTicket']) + '-' + str(document['gestion.fechaApertura'])}",
                "_source": filterKeys(document),
            }

In [53]:
try:
    concat = pd.merge(datos_SM_interactions,datos_semilla, on=['id_Beneficiario'],how='inner')
    concat = concat.rename(columns={'id_Beneficiario' : 'gestion.id_Beneficiario','resolution':'gestion.resolucion','variable3':'gestion.Tipo3','variable4':'gestion.Tipo4','nombreInstitucionEd':'gestion.nombreInstitucionEd', 'open':'gestion.abierto'
                                    ,'open_time':'gestion.fechaApertura','close_time':'gestion.fechaCierre','nombreSede':'gestion.nombreSede'
                                    ,'site_id':'gestion.site_id','nombre_departamento':'gestion.dptoGestion'
                                    ,'nombre_municipio':'gestion.muniGestion','nombre_centro_pob':'gestion.nombre_centro_pob'
                                    ,'COD_ISO':'gestion.COD_ISO','codDanesede':'gestion.codDanesede',
                                    'energiadesc':'gestion.energiadesc'
                                    ,'cod_servicio':'gestion.cod_servicio'
                  , 'codDaneMuni':'gestion.codDaneMuni', 'nombre_centro_pob':'gestion.nombre_centro_pob', 'codCentroPoblado':'gestion.codCentroPoblado'
                  , 'codDaneInstitucionEdu':'gestion.codDaneInstitucionEdu', 'tipoSitio':'gestion.tipoSitio', 'detalleSitio':'gestion.detalleSitio'
                  , 'energia':'gestion.energia', 'region':'gestion.region', 'matricula':'gestion.matricula', 'DDA':'gestion.DDA'
                  , 'grupoDesc':'gestion.grupoDesc','estadoInstalacion':'gestion.estadoInstalacion'                                                                   
                                   })
    concat['gestion.fechaApertura'] = concat['gestion.fechaApertura'].str.replace("T"," ")
    concat['gestion.fechaApertura'] = concat['gestion.fechaApertura'].str.slice(stop=19)
    concat['gestion.fechaCierre'] = concat['gestion.fechaCierre'].str.replace("T"," ")
    concat['gestion.fechaCierre'] = concat['gestion.fechaCierre'].str.slice(stop=19)
    concat["gestion.anyo"] = concat["gestion.fechaApertura"].str[0:4]
    concat["gestion.mes"] = concat["gestion.fechaApertura"].str[5:7]
    concat["gestion.dia"] = concat["gestion.fechaApertura"].str[8:10]
    concat["gestion.hora"] = concat["gestion.fechaApertura"].str[11:13]
    concat["gestion.min"] = concat["gestion.fechaApertura"].str[14:16]
    concat["gestion.seg"] = concat["gestion.fechaApertura"].str[17:19]
    concat['@timestamp'] = now.isoformat()
    salida = helpers.bulk(es, doc_generator(concat))
    print("Fecha: ", now,"- Interacciones insertadas en indice principal:",salida[0])
except Exception as e:
    print(e, "\nNinguna Interacción insertada en indice principal:")

    

Ninguna Interacción insertada en indice principal:


Actualizando fecha de control de ejecución

In [55]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    
#fecha_ejecucion = '2021-04-29T00:00:00'

if fecha_ejecucion > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'
response = es.index(
        index = parametros.mintic_control,
        id = 'jerarquia_gestion_interacciones',
        body = { 'jerarquia_gestion_interacciones': 'gestion_interacciones','gestion.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)


actualizada fecha control de ejecucion: 2021-07-01T18:00:00
